In [3]:
import sys
import math

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sklearn
import numpy as np

from delayedsparse import delayedsparse
from delayedsparse.ca import CA
from sklearn.preprocessing import StandardScaler

In [7]:
# load data
path = '/home/livtollanes/NewData/edgelist.csv'

req_cols = ['marker_id', 'follower_id', 'twitter_name']
dtypes = {'marker_id': 'object',
          'follower_id': 'object',
          'twitter_name': 'object'}


edgelist_CA = pd.read_csv(path, usecols=req_cols, dtype=dtypes)

In [10]:
#convert marker_id and follower_id to numeric
edgelist_CA['marker_id'] = pd.to_numeric(edgelist_CA['marker_id'])
edgelist_CA['follower_id'] = pd.to_numeric(edgelist_CA['follower_id'])


# sparse CA
Number_of_dimensions = 15


markers = edgelist_CA['marker_id'].unique()
followers = edgelist_CA['follower_id'].unique()

Number_of_markers = markers.size
Number_of_followers = followers.size

# Assign a unique number to each marker and follower - i.e. column positions
markers_dic = pd.Series(index=edgelist_CA['marker_id'].unique(),data=np.arange(Number_of_markers,dtype=int)) # these are column positions of markers
followers_dic = pd.Series(index=edgelist_CA['follower_id'].unique(),data=np.arange(Number_of_followers,dtype=int)) # these are column positions of followers

# merging row and col matrix indices into a new columgn in the bipartite graph dataframe
edgelist_CA['i'] = edgelist_CA['marker_id'].map(markers_dic) #i markers
edgelist_CA['j'] = edgelist_CA['follower_id'].map(followers_dic) #j followers

# dropgin repeated edges just in case
edgelist_CA.dropna(inplace=True)
edgelist_CA.shape

# creating a sparse matrix using the Coo (coordinate) format 
coo = coo_matrix((np.ones((edgelist_CA.shape[0])), (edgelist_CA['j'].values, edgelist_CA['i'])), shape=(Number_of_followers, Number_of_markers))
X = coo.tocsr()

In [81]:
# computing the sparse CA
ca = CAWithInertia(n_components=Number_of_dimensions)
ca.fit(X)
ca.compute_explained_inertia()

# extracting results
aug_inter = np.array(ca.explained_inertia_)


AttributeError: 'CAWithInertia' object has no attribute 'S'

In [70]:

dimensions_names = ['latent_dim_%d'%(d+1) for d in range(Number_of_dimensions)]


scaler = StandardScaler(with_std=True)

In [82]:
pwd

'/home/livtollanes/10.jan-thesis/Code'

In [71]:


scaler.fit(np.vstack(((ca.F*1)[:,:Number_of_dimensions],(ca.G*1)[:,:Number_of_dimensions])))   

StandardScaler(copy=True, with_mean=True, with_std=True)

In [72]:
followers_coords = pd.DataFrame(index=followers,columns=dimensions_names,data=scaler.transform((ca.F*1)[:,:Number_of_dimensions]))
markers_coords = pd.DataFrame(index=markers,columns=dimensions_names,data=scaler.transform((ca.G*1)[:,:Number_of_dimensions]))